In [1]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


seasons = 1
Game = Regency(IntDC=30)
clear_output()

for a in range(seasons):
    print(a)
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(Update=True)
    Game.take_domain_actions(None)
Game.clear_screen()
Game.Seasons[Game.Season]['Season']

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,GG,False,No Event,77,20,56,46,30,Rich
1,CA,False,No Event,34,20,2,8,6,Dormant
2,SM,False,No Event,15,19,0,0,0,Dormant
3,HA,False,No Event,28,19,13,22,9,Average
4,JR,False,Brigandage,33,19,-1,9,10,Dormant
5,WM,False,Natural Event,7,19,1,2,1,Dormant
6,TF,False,No Event,2,19,1,2,1,Dormant
7,Is,False,Monsters,106,19,10,20,10,Average
8,LF,False,Natural Event,56,18,3,9,14,Dormant
9,Sw,False,Corruption/Crime,11,18,0,0,0,Dormant


In [2]:
import pandas as pd
df = pd.concat([pd.concat([Game.Seasons[a]['Actions'][b+1] for b in range(3)]) for a in range(seasons)])

In [3]:
set(df['Action'])

{'Lieutenant',
 'agitiate_for_friend',
 'declare_war',
 'decree_asset_seizure',
 'decree_general',
 'diplomacy_trade_agreement',
 'disband_army',
 'disband_mercenaries',
 'espionage_assassination',
 'espionage_corruption',
 'espionage_diplomatic_details',
 'espionage_discover_troop_movements',
 'espionage_heresy',
 'espionage_intrigue',
 'grant',
 'muster_army',
 'muster_levies',
 'muster_mercenaries'}

In [14]:
self=Game
import pandas as pd
import numpy as np
Target = 'City of Anuire'
Base = 'Bogsend'
Regent = 'EC'
self.add_relationship('CD','EC', Trade_Permission=1)